In [2]:
using ODE
using PyPlot
using ProgressMeter

In [3]:
function f(t, v)
    (x1,x2,y1,y2) = v
    xpp = - R * x2 - C * x1
    ypp = - R * y2 - C * y1
    for i = 1:size(magnets)[1]
        xpp = xpp + (magnets[i,1] - x1) / ( (magnets[i,1] - x1)^2 + (magnets[i,2] - y1)^2 + d^2)^(3/2)
        ypp = ypp + (magnets[i,2] - y1) / ( (magnets[i,1] - x1)^2 + (magnets[i,2] - y1)^2 + d^2)^(3/2)
    end

    [x2;xpp;y2;ypp]
end

function closest(y)
    last_x = y[end][1]
    last_y = y[end][3]
    
    dists = [sqrt((magnets[i,1] - last_x)^2 + (magnets[i,2] - last_y)^2) for i = 1:size(magnets)[1]]
    min = minimum(dists)
    return find(dists .== min)[1]
end

closest (generic function with 1 method)

In [5]:
l = 0.5
magnets = [
    [l * sin(0.0/180.0 * pi) l * cos(0.0/180.0 * pi)];
    [l * sin(120.0/180.0 * pi) l * cos(120.0/180.0 * pi)];
    [l * sin(240.0/180.0 * pi) l * cos(240.0/180.0 * pi)]
]

R = 0.15
C = 0.07
d = 0.25


0.25

In [6]:
time = 0:0.01:5;
xv = collect(-1:0.002:1)
yv = collect(-1:0.002:1)

n = length(xv)
m = zeros(n,n)

p = Progress(n*n, 1)

fd = open("/Users/zahedi/Desktop/m.csv","w")

for xi = 1:n
    for yi = 1:n
        y0 = [xv[xi]; 0.0; yv[yi]; 0.0]
        t, y = ode45(f, y0, time);
        write(fd, "$(xv[xi]),$(yv[yi]),$(closest(y))\n")
        flush(fd)
        next!(p)
    end
end
close(fd)

Progress: 100% Time: 1:23:47


In [7]:
N = 100

m = readcsv("/Users/zahedi/Desktop/m.csv")

indices = zeros(N,3)
indices_index = 1

time = 0:0.01:10;

for i = 1:N
    a = [0 0 0]
    while a[3] != round(ceil(rand()*3),0)
        a = m[round(ceil(rand()*size(m)[1]),0),:]
    end
    indices[indices_index,:] = a
    indices_index = indices_index + 1
end

for i = 1:N
    y0 = [indices[i,1]; 0.0; indices[i,2]; 0.0]
    t, y = ode45(f, y0, time);    
    writecsv("/Users/zahedi/Desktop/run_$i.csv", [[v[1] for v in y] [v[3] for v in y]])
end